In [ ]:
.libPaths("/lustre/groups/itg/teams/zeggini/projects/T2D-diamante/T2DGGI_MR/Rpackages")
##Paths to specify by the user:
pathMR
setwd(pathMR)
pathMR_Results
pathMR_ARIC_EA
pathMR_ARIC_AA
pathMR_GENOA_AA
pathMR_GENOA_EA
path_T2DGGI
path_1kG #Path with bfiles from the 1000Genomes project
path_code #Path of the scripts with MR functions
path_Borges = #Path with summary statistics from Borges et al. 2022
pathMR_GALAII_PR =
pathMR_GALAII_MX =
pathMR_SAGE_AA =
pathMR_GTEx_EUR =
path_eQTLGen =

In [ ]:
library(data.table)
library(genetics.binaRies)
library(TwoSampleMR)

# Functions needed to run MR


## Examples of import functions for ARIC and UKB

In [ ]:
#Function to import data and get the right file names (adapted to ARIC data)
#Perform correction if needed
Get_data_ARIC <- function(import.args){
    #This function needs filename, pathdata and correction, qtl.file.out as extra.args
    #Also needs qtl.out.prefix and qtl.out.suffix to save file
    qtl <- fread(paste0(import.args[["path.qtl.in"]], import.args[["filename"]]))
    qtl <- qtl[, .(chr=`#CHROM`,
                     pos=POS,
                     pval=P,
                     beta=BETA,
                     se=SE,
                     eaf=ifelse(A1==REF, 1-A1_FREQ, A1_FREQ),
                     # n,
                     rsid=ID,
                     ea=ALT,
                     nea=REF)]
    #If the p-values need to be transformed, adjust using the given correction
    #In that case, need to rewrite the file for clumping (need to change the name so that it matches the other functions after)
    if(!is.na(import.args$correction)){
        qtl[,"pval"] <- p.adjust(unlist(qtl[,"pval"]), method = import.args[["correction"]])
        fwrite(qtl, paste0(import.args[["path.qtl.out"]], "forclumping.", import.args[["correction"]], ".", import.args[["filename"]]), sep = "\t")
    }else{
        fwrite(qtl, paste0(import.args[["path.qtl.out"]], "forclumping.", import.args[["filename"]]), sep = "\t")
    }
    qtl
}

## Clumping functions

In [ ]:
#Function to clump the data, check if the proxy is in T2DGGI datasets and compute Fstat
Clump_proxies_Fstat <- function(ref.bfile.path, qtl.path, qtl.name, pval.thr, r2_thr = 0.001, kb_thr = 10000, plink = "plink", rsID, pval, qtl, t2d, bfile.path){
    system(paste0("mkdir -p ", qtl.path, "Clumping/"))
    output.qtl <- data.table()

    #The firt arguments of the functions are for the clumping, and the last ones for the data
    #Clump the variants
    if(!file.exists(paste0(qtl.path, "Clumping/", qtl.name, ".clumped"))) system(paste0(plink, " --bfile ", ref.bfile.path, " --clump-p1 ", pval.thr, " --clump-r2 ", r2_thr, " --clump-kb ", kb_thr, " --clump ", qtl.path, qtl.name, " --clump-snp-field ", rsID, " --clump-field ", pval, " --out ", qtl.path, "Clumping/", qtl.name))
    #Import the clumping results
    #Need to check if there are variants left after clumping, not the case for example if only ine significant variant but not in reference data
    if(!file.exists(paste0(qtl.path, "Clumping/", qtl.name, ".clumped"))) return(output.qtl)

    clumping.res <- fread(paste0(qtl.path, "Clumping/", qtl.name, ".clumped"))
    clumped.ivs <- clumping.res$SNP

    #Compute F-stat
    qtl$snp_fstat <- (qtl$beta)^2/(qtl$se)^2

    #Only keep the variants that are clumped
    qtl[, iv:=ifelse(rsid %in% clumped.ivs & snp_fstat>=10, TRUE, FALSE)]

    #Check which clumped IVs are not available in T2D of matching ancestry
    need.proxy <- subset(qtl, iv)$rsid[which(!(subset(qtl, iv)$rsid %in% t2d$rs_id))]

    ## Find proxies if IV not included in T2D sumstats
    if(length(need.proxy)>0){
        proxies <- NULL
        iv=need.proxy[1]
        for (iv in need.proxy){
            cluster.snps <- unlist(strsplit(gsub("\\(1\\)", "", clumping.res[SNP==iv, SP2]), split=","))

            # Check LD between iv and potential.proxy
            ld.matrix <- ieugwasr::ld_matrix(c(iv, cluster.snps),
                                           plink_bin = genetics.binaRies::get_plink_binary(),
                                           bfile = ref.bfile.path)
            #Take the corresponding iv and remove the correlation with itself
            ld.vector <- ld.matrix[grep(rownames(ld.matrix), pattern = iv),-grep(rownames(ld.matrix), pattern = iv)]
            rm(ld.matrix)

            #Check if any of the proxies is in t2d
            all.potential.proxies = sub("_.*", "", names(which((ld.vector**2)>0.8)))
            if(nrow(subset(t2d[rs_id %in% all.potential.proxies]))>0){
                for (potential.proxy in sub("_.*", "", names(which((ld.vector**2)>0.8)))){
                # Check if potential.proxy is in T2D data
                if(nrow(t2d[rs_id==potential.proxy])!=0) break
                }
                proxies <- c(proxies, potential.proxy)
            }
        }

    ## Update IVs in sig.qtl
    qtl[rsid %in% need.proxy, iv:=FALSE]
    qtl[rsid %in% proxies, iv:=TRUE]

    }

    output.qtl <- rbind(output.qtl, qtl[iv==TRUE])
    output.qtl
}

## Wrapper to apply the different functions

In [ ]:
###Import the files and change the names
#Loop over molecular traits
#Information to be provided by the user:at leasat study.qtl,type of qtl and ancestry
wrap.findivs <- function(info.datasets, info.t2d, qtl.study, ancestry, qtl.type, function.import.qtl, import.extra.args = NULL, params.eQTL){
    #function.import.qtl is a function needed to import the data that handle the columns names
    #params.eQTL is needed if eQTL are analysed and contains the column names of the input file in a list with names:
    #list(chr, pos, pval, beta, se, eaf, ea, nea, rsid, mol.trait))
    info.study <- subset(info.datasets, STUDY == qtl.study & ANCESTRY == ancestry & QTL_TYPE == qtl.type)
    tissue = info.study$TISSUE
    build=info.study$BUILD
    correction = info.study$CORRECTION
    pval.thr = info.study$PVAL_THR
    ancestry_T2DGGI = info.study$ANCESTRY_T2D
    path.in.qtl = info.study$PATH_QTL
    prefix.in.qtl = info.study$PREFIX_QTL
    suffix.in.qtl = info.study$SUFFIX_QTL
    ivs.dt <- data.table::data.table()
    path.output = paste0(pathMR, "dataforMR/")
    # Read ancestry-specific T2D data
    info.t2d.ancestry <- subset(info.t2d, ancestry == ancestry_T2DGGI)
    t2d <- fread(paste0(path_T2DGGI, ancestry_T2DGGI, "_MetalFixed_LDSC-CORR_Results1TBL_rsid.gz"))
    t2d <- t2d[, .(chr.b38=as.integer(sub("chr", "", CHR_HG38)),
                 pos.b38=POS_HG38,
                 chr.b37=as.integer(sub("chr", "", CHR_HG19)),
                 pos.b37=POS_HG19,
                 rs_id=RSID,
                 p_value=`P-value`,
                 beta=Effect,
                 standard_error=StdErr,
                 effect_allele_frequency=Freq1,
                 n=info.t2d.ancestry$n.cases+info.t2d.ancestry$n.controls,
                 ncases=info.t2d.ancestry$n.cases,
                 effect_allele=toupper(Allele1),
                 other_allele=toupper(Allele2))]
    pathMR_anc = paste0(pathMR, "datasets/", qtl.type, "_", qtl.study, "_", ancestry, "/")
    #It is supposed that the summary statistics are in pathMR_anc
    #Create directory if does not exist
    system(paste0("mkdir -p ", pathMR_anc))

    #Name for the output files if pQTL or metaboQTL as multiple datasets will be concatenated
    if(qtl.type %in% c("pQTL", "metaboQTL")){
        list_files <- intersect(list.files(path.in.qtl, pattern = paste0("^", prefix.in.qtl)), list.files(path.in.qtl, pattern = paste0(suffix.in.qtl, "$")))
        #Loop over molecular traits to get the data and transform the p-values if needed
        data_allQTL <- lapply(list_files, function(mol.trait) function.import.qtl(import.args = c(import.extra.args, path.qtl.in = path.in.qtl, filename = mol.trait, path.qtl.out = pathMR_anc)))
        #Change the file names if correction and change only the mol trait name
        if(!is.na(correction)){
            names.qtls <- names(data_allQTL) <- paste0(unlist(sapply(list_files, function(z) gsub(z, pattern = paste0(prefix.in.qtl, "|", suffix.in.qtl), replacement = ""))))
            prefix.in.qtl <- paste0("fdr.", prefix.in.qtl)
        }else{
            names.qtls <- names(data_allQTL) <- paste0(unlist(sapply(list_files, function(z) gsub(z, pattern = paste0(prefix.in.qtl, "|", suffix.in.qtl), replacement = ""))))
        }
        #molecular traits that have a significant pQTL
        names.qtls.sig <- names(data_allQTL)[which(unlist(lapply(data_allQTL, function(qtl) min(qtl[,"pval"])<pval.thr)))]
        #Clump the data
        data_allQTL.Clump.Fstat10 <- lapply(names.qtls.sig, function(mol.trait){ tmp.file <- Clump_proxies_Fstat(ref.bfile.path = paste0(path_1kG, ancestry_T2DGGI),
                                        qtl.path = pathMR_anc, pval.thr = pval.thr, r2_thr = 0.001, kb_thr = 10000,
                                        qtl.name = paste0("forclumping.", prefix.in.qtl, mol.trait, suffix.in.qtl), plink = "plink",
                                        rsID = "rsid", pval = "pval", qtl = data_allQTL[[mol.trait]], t2d = t2d) ;
                                        tmp.file$mol.trait <- mol.trait ;
                                        tmp.file})
        names(data_allQTL.Clump.Fstat10) <- names.qtls.sig
    }else{
        #if eQTL, no need to loop over files
        qtl <- fread(paste0(pathMR_anc, prefix.in.qtl, ancestry, suffix.in.qtl))
        qtl <- setnames(qtl, old=c(params.eQTL$chr, params.eQTL$pos, params.eQTL$pval, params.eQTL$beta, params.eQTL$se, params.eQTL$eaf, params.eQTL$ea, params.eQTL$nea, params.eQTL$rsid, params.eQTL$mol.trait),
                             new = c("chr", "pos", "pval", "beta", "se", "eaf", "ea", "nea", "rsid", "mol.trait"))
        names.qtls <- unique(qtl$mol.trait)
        #Name of the molecular traits with at least one qtl
        names.qtls.sig <- unique(subset(qtl, pval < pval.thr)$mol.trait)
        #Create empty list to get the clump files
        data_allQTL.Clump.Fstat10 <- vector(length(names.qtls.sig), mode = "list")
        names(data_allQTL.Clump.Fstat10) <- names.qtls.sig
        #Loop only over the molecular traits with qtls
        for (cur.trait in names.qtls.sig){
            qtl.cur <- subset(qtl, mol.trait == cur.trait)
            #Export a tmp file with the corresponding gene
            fwrite(qtl.cur, file = paste0(pathMR_anc, cur.trait, ".txt.gz"), sep = "\t")
            data_allQTL.Clump.Fstat10[[cur.trait]] <- Clump_proxies_Fstat(ref.bfile.path = paste0(path_1kG, ancestry_T2DGGI),
                                qtl.path = pathMR_anc, pval.thr = pval.thr, r2_thr = 0.001, kb_thr = 10000,
                                qtl.name = paste0(cur.trait, ".txt.gz"), plink = "plink",
                                rsID = "rsid", pval = "pval", qtl = subset(qtl, mol.trait == cur.trait), t2d = t2d)
            #Give name of the trait only if there is an output from clumping
            if(nrow(data_allQTL.Clump.Fstat10[[cur.trait]])>0) data_allQTL.Clump.Fstat10[[cur.trait]]$mol.trait <- cur.trait
            system(paste0("rm ", pathMR_anc, cur.trait, ".txt.gz"))
        }
    }
    ##############REST OF THE FUNCTION IS COMMON TO ALL DATASETS
    #Check if some clumping gives no output (error from the clumping function)
    if(any(unlist(lapply(data_allQTL.Clump.Fstat10, function(z) nrow(z)==0)))){
        qtl.noclump <- names(which(unlist(lapply(data_allQTL.Clump.Fstat10, function(z) nrow(z) ==0))))
        #Remove these qtl from the list and from the qtl sig
        data_allQTL.Clump.Fstat10 <- data_allQTL.Clump.Fstat10[-which(names(data_allQTL.Clump.Fstat10) %in% qtl.noclump)]
        names.qtls.sig <- names.qtls.sig[-which(names.qtls.sig %in% qtl.noclump)]
    }

    #Rename to match the format of pQTL and metaboQTL
    names(data_allQTL.Clump.Fstat10) <- paste0(qtl.type, "__", qtl.study, "__", tissue, "__", names.qtls.sig, "__", ancestry, "__", build)

    #Make a dataframe with all IVs
    ivs.dt <- do.call(rbind, lapply(names(data_allQTL.Clump.Fstat10), function(name.qtl)
            unique(rbind(ivs.dt, data.table::data.table(rsid=data_allQTL.Clump.Fstat10[[name.qtl]][iv==TRUE, rsid],
                                                        study=qtl.study,
                                                        ancestry = ancestry,
                                                        tissue=tissue,
                                                        mol.trait=strsplit(name.qtl, split = "__")[[1]][4])))))
    #Add the molecular traits without any IVs, just to keep the information
    if(!(all(names.qtls %in% names.qtls.sig))){
        ivs.dt <- rbind(ivs.dt, data.table(rsid = "NO_IVs", study = qtl.study, ancestry = ancestry, tissue = tissue,
                                       mol.trait = unlist(sapply(names.qtls[-which(names.qtls %in% names.qtls.sig)], function(z) strsplit(z, split = "__")[[1]][4]))))
    }

    ## Output clumped QTL data for all significant molecular traits in qtl.study
    clumped.qtl <- do.call(rbind, data_allQTL.Clump.Fstat10)

    #Change the column names for MR functions
    clumped.qtl <- clumped.qtl[, .(chromosome=chr,
                                   base_pair_location = pos,
                                   p_value = pval,
                                   beta = beta,
                                   standard_error = se,
                                   effect_allele_frequency = eaf,
                                   rs_id = rsid,
                                   effect_allele = ea,
                                   other_allele = nea,
                                   snp_fstat = snp_fstat,
                                    mol.trait = mol.trait,
                                   n = info.study$N)]

    data.table::fwrite(clumped.qtl, paste0(path.output, "clumped__", qtl.type, "__", qtl.study, "__", tissue, "__", ancestry, "__", build, ".txt"))

    ## Prepare final T2D file containing all IVs for all QTL studies (can be done per study)
    #Keep the corresponding build
    t2d.ivs <- subset(t2d, rs_id %in% ivs.dt$rsid[grepl(ivs.dt$rsid, pattern = "rs")])
    if(build == "37"){
        t2d.ivs <- t2d.ivs[, c("chr.b38", "pos.b38"):=NULL]
        colnames(t2d.ivs)[1:2] <- c("chromosome", "base_pair_location")
    }else{
        t2d.ivs <- t2d.ivs[, c("chr.b37", "pos.b37"):=NULL]
        colnames(t2d.ivs)[1:2] <- c("chromosome", "base_pair_location")
    }
    data.table::fwrite(t2d.ivs, paste0(path.output, "clumped__T2D__", qtl.type, "__", qtl.study, "__", tissue, "__", ancestry, "__", build, ".txt.gz"))

    ## Output all IVs with columns: rsid, study, tissue and molecular trait
    #Export this file to have information on qtl without any IV
    data.table::fwrite(ivs.dt, paste0(path.output, "ivs__", qtl.type, "__", qtl.study, "__", tissue, "__", ancestry, "__", build, ".txt"))
}

# Information needed on the datasets to run the MR analysis

In [ ]:
##Dataframe with the different datasets to use
#"/lustre/groups/itg/shared/referenceData/sumstats/Borges_2022_UKBNMR/"
info.datasets.forMR <- data.frame(STUDY = c("ARIC", "ARIC", "GENOA", "GENOA", "UKB", "GALAII", "GALAII", "SAGE", "eQTLGen"),
                                 TISSUE = c("plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "plasma"),
                                 BUILD = c("38", "38", "37", "37", "37", "38", "38", "38", "37"),
                                 QTL_TYPE = c("pQTL", "pQTL", "eQTL", "eQTL", "metaboQTL", "eQTL", "eQTL", "eQTL", "eQTL"),
                                 CORRECTION = c("fdr", "fdr", NA, NA, NA, NA, NA, NA, NA),
                                  #Here we put 1 as pval thr for GALAII as results have already been filtered, same for eQTLGen
                                 PVAL_THR = c(0.05, 0.05, 5e-4, 5e-4, 5e-8, 1, 1, 1, 1),
                                 ANCESTRY = c("EA", "AA", "AA", "EA", "EUR", "PR", "MX", "AA", "EUR"),
                                 ANCESTRY_T2D = c("EUR", "AFR", "AFR", "EUR", "EUR", "AMR", "AMR", "AFR", "EUR"),
                                 N = c(7213, 1871, 1032, 801, 115078, 893, 784, 757, 31684),
                                 PATH_QTL = c(pathMR_ARIC_EA, pathMR_ARIC_AA, pathMR_GENOA_AA, pathMR_GENOA_EA, path_Borges, pathMR_GALAII_PR, pathMR_GALAII_MX, pathMR_SAGE_AA, path_eQTLGen),
                                 PREFIX_QTL = c("", "", "GENOA_", "GENOA_", "met-d-", "allchr.pairseQTLTRUE.more.galasage.", "allchr.pairseQTLTRUE.more.galasage.", "allchr.pairseQTLTRUE.more.galasage.", "2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe."),
                                 SUFFIX_QTL = c(".PHENO1.glm.linear.gz", ".PHENO1.glm.linear.gz", "_summary_statistics.txt.gz", "_summary_statistics.txt.gz", ".vcf.gz", ".hg38.txt.gz", ".hg38.txt.gz", ".hg38.txt.gz", ".txt.gz"))


In [ ]:
#Number of cases and controls in each of the ancestries (numbers obtained from Suzuki et al. 2024)
info.T2DGGI <- data.frame(ancestry = c("AFR", "EAS", "EUR", "AMR", "SAS"),
                         n.cases = c(50251, 88109, 242283, 29375, 16832),
                         n.controls = c(103909, 339395, 1569734, 59368, 33767))

# Apply to differnt datasets

In [ ]:
##Apply on different datasets
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "ARIC", ancestry = "EA", qtl.type = "pQTL", function.import.qtl = Get_data_ARIC, import.extra.args = list(correction = "fdr"))
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "GENOA", ancestry = "AA", qtl.type = "eQTL", params.eQTL = list(chr = "chr", pos = "ps", pval = "p_wald", beta = "beta", se = "se", eaf = "af", ea = "allele1", nea = "allele0", rsid = "rs", mol.trait = "GENE"))
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "GENOA", ancestry = "EA", qtl.type = "eQTL")
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "UKB", ancestry = "EUR", qtl.type = "metaboQTL", function.import.qtl = Get_metabo_UKB, import.extra.args = list(pval.thr = 5e-8))
##GALAII cohorts, including SAGE
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "GALAII", ancestry = "PR", qtl.type = "eQTL", params.eQTL = list(chr = "chr", pos = "pos", pval = "pval_nominal", beta = "slope", se = "slope_se", eaf = "maf", ea = "alt", nea = "ref", rsid = "variant_id", mol.trait = "gene_id"))
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "GALAII", ancestry = "MX", qtl.type = "eQTL", params.eQTL = list(chr = "chr", pos = "pos", pval = "pval_nominal", beta = "slope", se = "slope_se", eaf = "maf", ea = "alt", nea = "ref", rsid = "variant_id", mol.trait = "gene_id"))
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "SAGE", ancestry = "AA", qtl.type = "eQTL", params.eQTL = list(chr = "chr", pos = "pos", pval = "pval_nominal", beta = "slope", se = "slope_se", eaf = "maf", ea = "alt", nea = "ref", rsid = "variant_id", mol.trait = "gene_id"))
##GTEx
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "GTEx", ancestry = "EUR", qtl.type = "eQTL", params.eQTL = list(chr = "chr", pos = "pos", pval = "pval_nominal", beta = "slope", se = "slope_se", eaf = "maf", ea = "alt", nea = "ref", rsid = "variant_id", mol.trait = "gene_id"))
#eQTLGen
wrap.findivs(info.datasets.forMR, info.T2DGGI, qtl.study = "eQTLGen", ancestry = "EUR", qtl.type = "eQTL", params.eQTL = list(chr = "SNPChr", pos = "SNPPos", pval = "Pvalue", beta = "beta", se = "se", eaf = "Freq", ea = "AssessedAllele", nea = "OtherAllele", rsid = "SNP", mol.trait = "Gene"))


# MR
Useful comment on MR-PRESSO: https://github.com/rondolab/MR-PRESSO/issues/9

In [ ]:
#Import MR functions
#The last lines have been removed, corresponding to test of the code
source(paste0(path_code, "MR_functions.R"))
data.path <- paste0(pathMR, "dataforMR/")


In [ ]:
#Run on ARIC
res.dt <- wrap.MR(exposure.lst= "clumped_pQTL_ARIC_plasma_EA_38.txt",
                  t2d.filename="clumped_T2D_pQTL_ARIC_plasma_EUR_38.txt.gz",
                  ivs.path= paste0(data.path, "ivs_pQTL_ARIC_plasma_EA_38.txt"),
                  data.path=data.path,
                  output.file=paste0(pathMR_Results, "mr_results_pqtl_ARIC_plasma_EA.txt"))

In [ ]:
#Run on GENOA AA
system(paste0("mkdir -p ", pathMR, "harmonizedMR/eqtl_GENOA_plasma_AA/"))
res.dt <- wrap.MR(exposure.lst= "clumped__eQTL__GENOA__plasma__AA__37.txt",
                  t2d.filename="clumped__T2D__eQTL__GENOA__plasma__AFR__37.txt.gz",
                  ivs.path= paste0(data.path, "ivs__eQTL__GENOA__plasma__AA__37.txt"),
                  data.path=data.path, Radial = F,
                  output.file=paste0(pathMR_Results, "mr_results_eqtl_GENOA_plasma_AA.txt"),
                  output.harmonized = paste0(pathMR, "harmonizedMR/eqtl_GENOA_plasma_AA/eqtl_GENOA_plasma_AA_T2D_harmonized_"))

In [ ]:
#GENOA EA
system(paste0("mkdir -p ", pathMR, "harmonizedMR/eqtl_GENOA_plasma_EA/"))
res.dt <- wrap.MR(exposure.lst= "clumped__eQTL__GENOA__plasma__EA__37.txt",
                  t2d.filename="clumped__T2D__eQTL__GENOA__plasma__EA__37.txt.gz",
                  ivs.path= paste0(data.path, "ivs__eQTL__GENOA__plasma__EA__37.txt"),
                  data.path=data.path, Radial = F,
                  output.file=paste0(pathMR_Results, "mr_results_eqtl_GENOA_plasma_EA.txt"),
                  output.harmonized = paste0(pathMR, "harmonizedMR/eqtl_GENOA_plasma_EA/eqtl_GENOA_plasma_EA_T2D_harmonized_"))

In [ ]:
#GALAII PR
system(paste0("mkdir -p ", pathMR, "harmonizedMR/eqtl_GALAII_plasma_PR/"))
res.dt <- wrap.MR(exposure.lst= "clumped__eQTL__GALAII__plasma__PR__38.txt",
                  t2d.filename="clumped__T2D__eQTL__GALAII__plasma__PR__38.txt.gz",
                  ivs.path= paste0(data.path, "ivs__eQTL__GALAII__plasma__PR__38.txt"),
                  data.path=data.path,
                  output.file=paste0(pathMR_Results, "mr_results_eqtl_GALAII_plasma_PR.txt"),
                  output.harmonized = paste0(pathMR, "harmonizedMR/eqtl_GALAII_plasma_PR/eqtl_GALAII_plasma_PR_T2D_harmonized_"),
                 Radial = F)

In [ ]:
#GALAII MX
system(paste0("mkdir -p ", pathMR, "harmonizedMR/eqtl_GALAII_plasma_MX/"))
res.dt <- wrap.MR(exposure.lst= "clumped__eQTL__GALAII__plasma__MX__38.txt",
                  t2d.filename="clumped__T2D__eQTL__GALAII__plasma__MX__38.txt.gz",
                  ivs.path= paste0(data.path, "ivs__eQTL__GALAII__plasma__MX__38.txt"),
                  data.path=data.path, Radial = F,
                  output.file=paste0(pathMR_Results, "mr_results_eqtl_GALAII_plasma_MX.txt"),
                  output.harmonized = paste0(pathMR, "harmonizedMR/eqtl_GALAII_plasma_MX/eqtl_GALAII_plasma_MX_T2D_harmonized_"))

In [ ]:
#SAGE AA
system(paste0("mkdir -p ", pathMR, "harmonizedMR/eqtl_SAGE_plasma_AA/"))
res.dt <- wrap.MR(exposure.lst= "clumped__eQTL__SAGE__plasma__AA__38.txt",
                  t2d.filename="clumped__T2D__eQTL__SAGE__plasma__AA__38.txt.gz",
                  ivs.path= paste0(data.path, "ivs__eQTL__SAGE__plasma__AA__38.txt"),
                  data.path=data.path, Radial = F,
                  output.file=paste0(pathMR_Results, "/mr_results_eqtl_SAGE_plasma_AA.txt"),
                  output.harmonized = paste0(pathMR, "harmonizedMR/eqtl_SAGE_plasma_AA/eqtl_SAGE_plasma_AA_T2D_harmonized_"))

In [ ]:
#eQTLGen EUR
system(paste0("mkdir -p ", pathMR, "harmonizedMR/eqtl_eQTLGen_plasma_EUR/"))
res.dt <- wrap.MR(exposure.lst= "clumped__eQTL__eQTLGen__plasma__EUR__37.txt",
                  t2d.filename="clumped__T2D__eQTL__eQTLGen__plasma__EUR__37.txt.gz",
                  ivs.path= paste0(data.path, "ivs__eQTL__eQTLGen__plasma__EUR__37.txt"),
                  data.path=data.path, Radial = F,
                  output.file=paste0(pathMR_Results, "/mr_results_eqtl_eQTLGen_plasma_EUR.txt"),
                  output.harmonized = paste0(pathMR, "harmonizedMR/eqtl_eQTLGen_plasma_EUR/eqtl_eQTLGen_plasma_EUR_T2D_harmonized_"))